In [ ]:
! pip install streamlit==0.84.2
! pip install streamlit_option_menu

In [ ]:
%%writefile dashbord.py
import math
import scipy
from scipy.optimize import fsolve
from scipy.interpolate import interp2d
from sympy import * 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import streamlit as st
import random
import plotly.express as px
from streamlit_option_menu import option_menu

floor_height = 3.2   # in metre
no_story = 10
terrain_category = 2    # City Centre
width = 15        # in metre
length = 15      # in metre

total_height = floor_height * no_story
tributory__area = floor_height * length
ratio1 = width/length       # ratio1 = a/b
ratio2 = total_height/length    # ratio2 = h/b

height = [0,3.2,6.4,9.6,12.8,16,19.2,22.4,25.6,28.8,32]
tributory_area = [24,48,48,48,48,48,48,48,48,48,24]

# Vb = basic wind speed
Vb = 33     # IS code 875 Part III 2015 -> Section 6.2 -> Figure 1

# Probability Factor = K1
K1 = 1      # IS code 875 Part III 2015 -> Section 6.3.1 -> Table 1

# Terrain and Height Factor = k2     
# IS code 875 Part III 2015 -> Section 6.3.2.2 -> Table 2
K2 = list(range(len(height)))
for i in K2:
  if height[i] < 10:
    height_std = [10, 15, 20, 30, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    terrain_category_std = [1,2,3,4]
    K2_std = np.array([(1.05,1.00,0.91,0.80), (1.09,1.05,0.97,0.80),
                        (1.12,1.07,1.01,0.80), (1.15,1.12,1.06,0.97),
                        (1.20,1.17,1.12,1.10), (1.26,1.24,1.20,1.20),
                        (1.30,1.28,1.24,1.24), (1.32,1.30,1.27,1.27),
                        (1.34,1.32,1.29,1.28), (1.35,1.34,1.31,1.30),
                        (1.37,1.36,1.32,1.31), (1.38,1.37,1.34,1.32),
                        (1.39,1.38,1.35,1.33), (1.40,1.39,1.36,1.34)])
  
    f = interp2d( terrain_category_std,height_std, K2_std, kind='linear', fill_value=-1)
    K2[i] =  f(terrain_category, 10)
  
  else :
    height_std = [10, 15, 20, 30, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    terrain_category_std = [1,2,3,4]
    K2_std = np.array([(1.05,1.00,0.91,0.80), (1.09,1.05,0.97,0.80),
                               (1.12,1.07,1.01,0.80), (1.15,1.12,1.06,0.97),
                               (1.20,1.17,1.12,1.10), (1.26,1.24,1.20,1.20),
                               (1.30,1.28,1.24,1.24), (1.32,1.30,1.27,1.27),
                               (1.34,1.32,1.29,1.28), (1.35,1.34,1.31,1.30),
                               (1.37,1.36,1.32,1.31), (1.38,1.37,1.34,1.32),
                               (1.39,1.38,1.35,1.33), (1.40,1.39,1.36,1.34)])
  
    f = interp2d( terrain_category_std,height_std, K2_std, kind='linear', fill_value=-1)
    K2[i] =  f(terrain_category, height[i])

# Topography Factor  = k3
# Level Ground
K3 = 1    # IS code 875 Part III 2015 -> Section 6.3.3.1 

# Cyclonic Importance Factor = k4
K4 = 1.3    # IS code 875 Part III 2015 -> Section 6.3.4

# Design Wind Speed (Vz)
Vz = list(range(len(height)))
for i in Vz :
  Vz[i] = Vb * K1 * K2[i] * K3 * K4     

# Wind directionality factor = Kd 
Kd = 0.9        # IS code 875 Part III 2015 -> Section 7.2.1

# Area averaging factor = Ka 
# IS code 875 Part III 2015 -> Section 7.2.2 -> Table 4
Ka = list(range(len(height)))
for i in range(len(height)):
  if tributory_area[i] <= 10 :
    Ka[i] = 1
  elif tributory_area[i] >= 100 :
    Ka[i] = 0.8
  else :
    area_std = [10,25,100]
    Ka_std = [1.0, 0.9, 0.8]
    Ka[i] = np.interp(tributory_area[i], area_std, Ka_std)

# Combination factor = Kc
Kc =  1    # IS code 875 Part III 2015 -> Section 7.3.3.13

# Design Wind Pressure = Pdz
Pz = list(range(len((height))))
Pdz  = list(range(len(height)))
for i in Pz:
  Pz[i] = 0.6 * Vz[i]**2    # Wind Pressure = Pz
  Pdz[i] = Kd * Ka[i] * Kc * Pz[i]

# Force Coefficient (drag coefficient) = Cf
# IS code 875 Part III 2015 -> Section 7.4.2 -> Fig 4 (a)
ratio1_std = [0, 0.25, 0.50, 0.75, 1, 1.25, 1.50, 1.75, 2, 2.25, 2.5, 2.75, 3]
ratio2_std = [0.25, 0.5, 1.0, 3.0, 5.0, 10, 20]
Cf_std = np.array([(1.200, 1.144, 1.176, 1.250, 1.330, 1.500, 1.750), 
                     (1.200, 1.161, 1.219, 1.301, 1.384, 1.685, 2.026),
                     (1.200, 1.178, 1.261, 1.352, 1.438, 1.869, 2.301),
                     (1.200, 1.183, 1.266, 1.362, 1.451, 1.888,	2.397),
                     (1.200, 1.148,	1.227, 1.307,	1.392, 1.784,	2.188),
                     (1.200, 1.096,	1.168, 1.248,	1.330, 1.654,	1.975),
                     (1.164, 1.043,	1.108, 1.188,	1.267, 1.523,	1.761),
                     (1.129, 1.000,	1.057, 1.143,	1.216, 1.427,	1.622),
                     (1.094, 0.957,	1.006, 1.097,	1.165, 1.330,	1.483),
                     (1.071, 0.931,	0.966, 1.066,	1.134, 1.271,	1.392),
                     (1.047, 0.915,	0.926, 1.034,	1.102, 1.211,	1.301),
                     (1.024, 0.905,	0.901, 1.000,	1.077, 1.168,	1.262),
                     (1.000, 0.899,	0.875, 0.966,	1.052, 1.125,	1.222)])
f = interp2d( ratio2_std, ratio1_std, Cf_std, kind='linear', fill_value=1)
Cf= f(ratio2, ratio1)

# Frontal Area
Af = list(range(len(height)))
for i in Af :
  if i==0 or i==10:
    Af[i] = 24
  else :
    Af[i] = 48

# wind Force
Fz = list(range(len(height)))
for i in Fz :
  Fz[i] = Cf * Af[i] * Pdz[i] /1000

with st.sidebar:
  selected = option_menu(
    menu_title = None,
    options = ["HOME","DATA", "OUTPUT", "CALCULATION", "SOURCE"],
    icons = ["house-fill","display", "display" ,"calculator-fill", "server"],
  )

if selected == "HOME" :
  st.header('Welcome')
  st.write('-----')
  st.subheader('Objective')
  st.write("Computing the lateral wind forces to be used in the design of the building at the various floor levels for a proposed 10-storey school building (with square plan) in Bengaluru")
  st.write("assuming the ‘general’ class building, ‘Category 2’ terrain and level ground ")
  


if selected == "DATA" :
  st.write(f"{selected}")
  st.write("---")
  
  Data = {
      "VARIABLE" : ["Floor height","No of stories","Terrain category","Width","Length","Total height","Tributory area","a/b","h/b" ],
      "VALUE" : [floor_height,no_story,terrain_category,width,length,total_height,tributory_area,ratio1,ratio2],
      "UNIT" : ["m"," "," ","m","m","m","m^2"," "," "],
  }
  df = pd.DataFrame(Data)
  st.dataframe(df)

if selected == "OUTPUT" :
  st.write(f"{selected}")
  st.write("-----")

  option = st.selectbox("Select",("Design Wind Speed","Design Wind Force","Design Wind Pressure"))
  if option == 'Design Wind Pressure' :
    Pressure = {
      "height(m)" : [height[0],height[1],height[2],height[3],height[4],height[5],height[6],height[7],height[8],height[9],height[10]],
      "Kd" : [Kd,Kd,Kd,Kd,Kd,Kd,Kd,Kd,Kd,Kd,Kd],
      "Ka" : [Ka[0],Ka[1],Ka[2],Ka[3],Ka[4],Ka[5],Ka[6],Ka[7],Ka[8],Ka[9],Ka[10]],
      "Kc" : [Kc,Kc,Kc,Kc,Kc,Kc,Kc,Kc,Kc,Kc,Kc],
      "Vz(m/s)" : [Vz[0],Vz[1],Vz[2],Vz[3],Vz[4],Vz[5],Vz[6],Vz[7],Vz[8],Vz[9],Vz[10]],
      "Pz(N/m^2)" : [Pz[0],Pz[1],Pz[2],Pz[3],Pz[4],Pz[5],Pz[6],Pz[7],Pz[8],Pz[9],Pz[10]],
      "Pdz(N/m^2)" : [Pdz[0],Pdz[1],Pdz[2],Pdz[3],Pdz[4],Pdz[5],Pdz[6],Pdz[7],Pdz[8],Pdz[9],Pdz[10]]
    }
    df = pd.DataFrame(Pressure)
    st.dataframe(df)

  if option == 'Design Wind Force' :
    Force = {
      "height(m)" : [height[0],height[1],height[2],height[3],height[4],height[5],height[6],height[7],height[8],height[9],height[10]],
      "Pdz(N/m^2)" : [Pdz[0],Pdz[1],Pdz[2],Pdz[3],Pdz[4],Pdz[5],Pdz[6],Pdz[7],Pdz[8],Pdz[9],Pdz[10]],
      "Cf" : [Cf,Cf,Cf,Cf,Cf,Cf,Cf,Cf,Cf,Cf,Cf],
      "Af(m^2)" : [Af[0],Af[1],Af[2],Af[3],Af[4],Af[5],Af[6],Af[7],Af[8],Af[9],Af[10]],
      "Fz(KN)" : [Fz[0],Fz[1],Fz[2],Fz[3],Fz[4],Fz[5],Fz[6],Fz[7],Fz[8],Fz[9],Fz[10]]
    }
    df = pd.DataFrame(Force)
    st.dataframe(df)  


  if option == 'Design Wind Speed' :
    velocity = { 
      "height(m)" : [height[0],height[1],height[2],height[3],height[4],height[5],height[6],height[7],height[8],height[9],height[10]],
      "K1" : [K1,K1,K1,K1,K1,K1,K1,K1,K1,K1,K1],
      "K2" : [K2[0],K2[1],K2[2],K2[3],K2[4],K2[5],K2[6],K2[7],K2[8],K2[9],K2[10]],
      "K3" : [K3,K3,K3,K3,K3,K3,K3,K3,K3,K3,K3],
      "K4" : [K4,K4,K4,K4,K4,K4,K4,K4,K4,K4,K4],
      "Vb(m/s)" : [Vb,Vb,Vb,Vb,Vb,Vb,Vb,Vb,Vb,Vb,Vb],
      "Vz(m/s)" : [Vz[0],Vz[1],Vz[2],Vz[3],Vz[4],Vz[5],Vz[6],Vz[7],Vz[8],Vz[9],Vz[10]]
    }
    df = pd.DataFrame(velocity)
    st.dataframe(df)


  

if selected == "CALCULATION" :
  st.write(f"{selected}","For First Node")
  st.write("-----")
  st.write("height =", height[1],"m")
  st.write("Tributory Area = ",str(tributory__area)[1],"mm^2")
  st.write("Basic Wind Speed(Vb) =",Vb, "m/s       (IS code 875 Part III 2015 -> Section 6.2 -> Figure 1)")
  st.write("Probability Factor(K1) =" ,K1,"        (IS code 875 Part III 2015 -> Section 6.3.1 -> Table 1)")
  st.write("Terrain and Height Factor(k2) =",int(K2[1]),"   (IS code 875 Part III 2015 -> Section 6.3.2.2 -> Table 2)")
  st.write("Topography Factor (k3) =",K3, "  for Level Ground     (# IS code 875 Part III 2015 -> Section 6.3.3.1 ) ")
  st.write("Cyclonic Importance Factor(k4)",K4,"     (IS code 875 Part III 2015 -> Section 6.3.4)")
  st.write("Design Wind Speed(Vz) = ",int(Vz[1]),"m/s" )
  st.write("Wind directionality factor(Kd) = ",Kd,"      (IS code 875 Part III 2015 -> Section 7.2.1)")
  st.write("Area averaging factor(Ka) = ",Ka[1],"        (IS code 875 Part III 2015 -> Section 7.2.2 -> Table 4)")
  st.write("Combination factor(Kc) = ",Kc,"             (IS code 875 Part III 2015 -> Section 7.3.3.13)")
  st.write("Design Wind Pressure(Pdz) = ",int(Pdz[1]),"N/m^2")
  st.write("Force Coefficient(Cf) = ",float(Cf),"        (IS code 875 Part III 2015 -> Section 7.4.2 -> Fig 4 (a))")
  st.write("Frontal Area(Af) = ",Af[1],"mm^2")
  st.write("wind Force(Fz) = ",float(Fz[1]),"KN")


if selected == "SOURCE" :
  st.write(f"{selected}")
  st.write("---")
  st.write("Is Code 875 Part III 2015")
  st.write("[Learn More >](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwibi4mqtJ_6AhVc3jgGHTvoAkMQFnoECAcQAQ&url=https%3A%2F%2Ftechnicalcivil.com%2Fwp-content%2Fuploads%2F2019%2F04%2FIS-875-part-3-2015.pdf%3Fx57228&usg=AOvVaw2WL9cYP5LzoHizOQj2JlTw)")



In [ ]:
! streamlit run dashbord.py & npx localtunnel --port 8501